In [5]:
from z3 import *

In [8]:
x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 9, x = 4]


In [11]:
#how to use the Z3 formula/expression simplifier. 
print (simplify(x + y + 2*x + 3))
print (simplify(x < y + x + 2))
print (simplify(And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5)))

3 + 3*x + y
Not(y <= -2)
And(x >= 2, 2*x**2 + y**2 >= 3)


In [13]:
x = Int('x')
y = Int('y')
print (x**2 + y**2 >= 1)
set_option(html_mode=False)
print (x**2 + y**2 >= 1)

x**2 + y**2 >= 1
x**2 + y**2 >= 1


In [16]:
#Z3 provides functions for traversing expressions.

x = Int('x')
y = Int('y')
n = x + y >= 3
print ("num args: ", n.num_args())
print ("children: ", n.children())
print ("1st child:", n.arg(0))
print ("2nd child:", n.arg(1))
print ("operator: ", n.decl())
print ("op name:  ", n.decl().name())



num args:  2
children:  [x + y, 3]
1st child: x + y
2nd child: 3
operator:  >=
op name:   >=


In [17]:
#solving non linear constraints
x = Real('x')
y = Real('y')
solve(x**2 + y**2 > 3, x**3 + y < 5)


[x = 1/8, y = 2]


In [23]:
#displaying in decimal notation
x = Real('x')
y = Real('y')
solve(x**2 + y**2 == 3, x**3 == 2)

set_option(precision=30)
print ("Solving, and displaying result with 30 decimal places")
solve(x**2 + y**2 == 3, x**3 == 2)

[x = 1.2599210498?, y = -1.1885280594?]
Solving, and displaying result with 30 decimal places
[x = 1.259921049894873164767210607278?,
 y = -1.188528059421316533710369365015?]


In [24]:
#rational numbers in Z3
print (1/3)          #not Z3 rational number 
print (RealVal(1)/3) #Z3 rational
print (Q(1,3))       #Z3 rational

x = Real('x')
print (x + 1/3)
print (x + Q(1,3))
print (x + "1/3")
print (x + 0.25)


0.3333333333333333
1/3
1/3
x + 3333333333333333/10000000000000000
x + 1/3
x + 1/3
x + 1/4


In [25]:
# Rational numbers can also be displayed in decimal notation.

x = Real('x')
solve(3*x == 1)

set_option(rational_to_decimal=True)
solve(3*x == 1)

set_option(precision=30)
solve(3*x == 1)



[x = 1/3]
[x = 0.333333333333333333333333333333?]
[x = 0.333333333333333333333333333333?]


In [26]:
# A system of constraints may not have a solution. In this case, we say the system is unsatisfiable.

x = Real('x')
solve(x > 4, x < 0)



no solution


---

---

# Boolean Logic

In [29]:
# Z3 supports Boolean operators: And, Or, Not, Implies (implication), If (if-then-else). 
# Bi-implications are represented using equality ==. 
# The following example shows how to solve a simple set of Boolean constraints.

p = Bool('p')
q = Bool('q')
r = Bool('r')
solve(Implies(p, q), r == Not(q), Or(Not(p), r))



[p = False, q = True, r = False]


In [31]:
# The Python Boolean constants True and False can be used to build Z3 Boolean expressions.

p = Bool('p')
q = Bool('q')
print (And(p, q, True))
print (simplify(And(p, q, True)))
print (simplify(And(p, False)))


And(p, q, True)
And(p, q)
False


In [32]:
#The following example uses a combination of polynomial and Boolean constraints.

p = Bool('p')
x = Real('x')
solve(Or(x < 5, x > 10), Or(p, x**2 == 2), Not(p))


[x = -1.414213562373095048801688724209?, p = False]


---

# Solvers

In [46]:
# Z3 provides different solvers,  for Eg.  solve in previous examples is implemented using z3 solver API
# Basic solver example 
x = Int('x')
y = Int('y')

#The command Solver() creates a general purpose solver. 
s = Solver()
print (s)

# Constraints can be added using the method add.
# We say the constraints have been asserted in the solver. 
s.add(x > 10, y == x + 2)
print (s)

# The method check() solves the asserted constraints.
print ("Solving constraints in the solver s ...")
print (s.check())

print ("Create a new scope...")
s.push()
s.add(y < 11)
print (s)
print ("Solving updated set of constraints...")
print (s.check())

print ("Restoring state...")
s.pop()
print (s)
print ("Solving restored set of constraints...")
print (s.check())


[]
[x > 10, y == x + 2]
Solving constraints in the solver s ...
sat
Create a new scope...
[x > 10, y == x + 2, y < 11]
Solving updated set of constraints...
unsat
Restoring state...
[x > 10, y == x + 2]
Solving restored set of constraints...
sat


In [48]:
#The following example shows an example that Z3 cannot solve

x = Real('x')
s = Solver()
s.add(2**x == 3)
print (s.check())

unknown


In [52]:
# traversing the constraints asserted into a solver
x = Real('x')
y = Real('y')
s = Solver()
s.add(x > 1, y > 1, Or(x + y > 3, x - y < 2))
print ("asserted constraints...")
for c in s.assertions():
    print (c)

asserted constraints...
x > 1
y > 1
Or(x + y > 3, x - y < 2)


In [53]:
# Collecting performance statistics for the check method. 
print (s.check())
print ("statistics for the last check method...")
print (s.statistics())
# Traversing statistics
for k, v in s.statistics():
    print ("%s : %s" % (k, v))

sat
statistics for the last check method...
(:arith-make-feasible 3
 :arith-max-columns   8
 :arith-max-rows      2
 :arith-rows          12
 :arith-upper         4
 :decisions           2
 :final-checks        1
 :max-memory          3.16
 :memory              2.87
 :mk-bool-var         10
 :num-allocs          1415119226
 :num-checks          1
 :rlimit-count        15075)
mk bool var : 1
decisions : 2
final checks : 1
num checks : 1
mk bool var : 5
arith-upper : 4
arith-rows : 12
arith-make-feasible : 3
arith-max-columns : 8
arith-max-rows : 2
mk bool var : 1
mk bool var : 1
mk bool var : 1
mk bool var : 1
num allocs : 1415119226
rlimit count : 15075
max memory : 3.16
memory : 2.87


---
# Arithmetic

In [55]:
#  A model is an interpretation that makes each asserted constraint true
#  The following example shows the basic methods for inspecting models. 

# Reals('x y z') creates the variables. x, y and z.
x, y, z = Reals('x y z')
s = Solver()
s.add(x > 1, y > 1, x + y > 3, z - x < 10)
print (s.check())

# The expression m[x] returns the interpretation of x in the model m
m = s.model()
print ("x = %s" % m[x])

# The expression "%s = %s" % (d.name(), m[d]) returns a string where the first %s is replaced with the name of d (i.e., d.name())
# and the second %s with a textual representation of the interpretation of d (i.e., m[d]).
print ("traversing model...")
for d in m.decls():
    print ("%s = %s" % (d.name(), m[d]))

sat
x = 1.5
traversing model...
z = 0
y = 2
x = 1.5


In [61]:
# Z3 supports real and integer variables. They can be mixed in a single problem.
# Z3Py will automatically add coercions converting integer expressions to real ones when needed
# different ways to declare integer and real variables.
x = Real('x')
y = Int('y')
a, b, c = Reals('a b c')
s, r = Ints('s r')
print (x + y + 1 + (a + s))

#The function ToReal casts an integer expression into a real expression.
print (ToReal(y) + c)

x + ToReal(y) + 1 + a + ToReal(s)
ToReal(y) + c


In [62]:
# Z3Py supports all basic arithmetic operations.
a, b, c = Ints('a b c')
d, e = Reals('d e')
solve(a > b + 2,
      a == 2*c + 10,
      c + b <= 1000,
      d >= e)

[b = 0, c = 0, e = 0, d = 0, a = 10]


In [70]:
# The command simplify applies simple transformations on Z3 expressions.
x, y = Reals('x y')
# Put expression in sum-of-monomials form
t = simplify((x + y)**3)
print (t)
t = simplify((x + y)**3, som=True)
print (t)

# Use power operator
t = simplify(t, mul_to_power=True)
print (t)


(x + y)**3
x*x*x + 3*x*x*y + 3*x*y*y + y*y*y
x**3 + 3*x**2*y + 3*x*y**2 + y**3


In [77]:
#Z3Py allows users to write option in two styles.

# 1  The Z3 internal option names start with : and words are separated by -
x, y = Reals('x y')
# Using Z3 native option names
print (simplify(x == y + 2, ':arith-lhs', True))

# 2 Z3Py also supports Python-like names, where : is suppressed and - is replaced with _
# Using Z3Py option names
print (simplify(x == y + 2, arith_lhs=True))

# command help_simplify() prints all available options
print ("\nAll available options:")
help_simplify()


x + -1*y == 2
x + -1*y == 2

All available options:
algebraic_number_evaluator (bool) simplify/evaluate expressions containing (algebraic) irrational numbers. (default: true)
arith_ineq_lhs (bool) rewrite inequalities so that right-hand-side is a constant. (default: false)
arith_lhs (bool) all monomials are moved to the left-hand-side, and the right-hand-side is just a constant. (default: false)
bit2bool (bool) try to convert bit-vector terms of size 1 into Boolean terms (default: true)
blast_distinct (bool) expand a distinct predicate into a quadratic number of disequalities (default: false)
blast_distinct_threshold (unsigned int) when blast_distinct is true, only distinct expressions with less than this number of arguments are blasted (default: 4294967295)
blast_eq_value (bool) blast (some) Bit-vector equalities into bits (default: false)
bv_extract_prop (bool) attempt to partially propagate extraction inwards (default: false)
bv_ineq_consistency_test_max (unsigned int) max size of c

---

# Functions

In [82]:
# function and constant symbols in pure first-order logic are uninterpreted or free
#  f applied twice to x results in x again, but f applied once to x is different from x.
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
solve(f(f(x)) == x, f(x) == y, x != y)
# The solution (interpretation) for f should be read as f(0) is 1, f(1) is 0, and f(a) is 1 for all a different from 0 and 1. 

#  we can also evaluate expressions in the model for a system of constraints. 
s = Solver()
s.add(f(f(x)) == x, f(x) == y, x != y)
print (s.check())
m = s.model()
print ("f(f(x)) =", m.evaluate(f(f(x))))
print ("f(x)    =", m.evaluate(f(x)))

[x = 0, y = 1, f = [1 -> 0, else -> 1]]
sat
f(f(x)) = 0
f(x)    = 1


---
# Satisfiability and Validity

In [88]:
# The following example redefines the Z3Py function 'prove' that receives a formula as a parameter.
# This function creates a solver, adds/asserts the negation of the formula, and check if the negation is unsatisfiable

p, q = Bools('p q')
demorgan = And(p, q) == Not(Or(Not(p), Not(q)))
print (demorgan)

def prove(f):
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
        print ("proved")
    else:
        print ("failed to prove")

print ("Proving demorgan...")
prove(demorgan)



And(p, q) == Not(Or(Not(p), Not(q)))
Proving demorgan...
proved


---
# List Comprehensions

In [146]:
#  They can be used to create Z3 expressions and problems in Z3Py.
#  The following example demonstrates how to use Python list comprehensions in Z3Py.
#  Create list [1, ..., 5] 
print ([ x + 1 for x in range(5) ])
# Create two lists containing 5 integer variables
X = [ Int('x%s' % i) for i in range(5) ]
Y = [ Int('y%s' % i) for i in range(5) ]
print (X)
print (Y)
# Create a list containing X[i]+Y[i]
X_plus_Y = [ X[i] + Y[i] for i in range(5) ]
print (X_plus_Y)
# # Create a list containing X[i] > Y[i]
X_gt_Y = [ X[i] > Y[i] for i in range(5) ]
print (X_gt_Y)

print (And(X_gt_Y))

# Create a 3x3 "matrix" (list of lists) of integer variables
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(3) ]
      for i in range(3) ]
pp(X) # pp is similar to print, but it uses Z3Py formatter for lists and tuples instead of Python's formatter.

# Z3Py also provides functions for creating vectors of Boolean, Integer and Real variables. These functions are implemented using list comprehensions.
X = IntVector('x', 5)
Y = RealVector('y', 5)
P = BoolVector('p', 5)
print (X)
print (Y)
print (P)
print ([ y**2 for y in Y ])
print (Sum([ y**2 for y in Y ]))


[1, 2, 3, 4, 5]
[x0, x1, x2, x3, x4]
<class 'z3.z3.ArithRef'>
[y0, y1, y2, y3, y4]
[x0 + y0, x1 + y1, x2 + y2, x3 + y3, x4 + y4]
[x0 > y0, x1 > y1, x2 > y2, x3 > y3, x4 > y4]
And(x0 > y0, x1 > y1, x2 > y2, x3 > y3, x4 > y4)
[[x_1_1, x_1_2, x_1_3],
 [x_2_1, x_2_2, x_2_3],
 [x_3_1, x_3_2, x_3_3]]
[x__0, x__1, x__2, x__3, x__4]
[y__0, y__1, y__2, y__3, y__4]
[p__0, p__1, p__2, p__3, p__4]
[y__0**2, y__1**2, y__2**2, y__3**2, y__4**2]
y__0**2 + y__1**2 + y__2**2 + y__3**2 + y__4**2


---
# Exercises

In [99]:
# Kinematic Equations
# In high school, students learn the kinematic equations. 
# These equations describe the mathematical relationship between displacement (d), time (t), acceleration (a), initial velocity (v_i) and final velocity (v_f).
# In Z3Py notation, we can write these equations as : d == v_i * t + (a*t**2)/2,v_f == v_i + a*t

# Problem 1
# Ima Hurryin is approaching a stoplight moving with a velocity of 30.0 m/s.
# The light turns yellow, and Ima applies the brakes and skids to a stop.
# If Ima's acceleration is -8.00 m/s2, then determine the displacement of the car during the skidding process.
d, a, t, v_i, v_f = Reals('d a t v__i v__f')
equations = [
   d == v_i * t + (a*t**2)/2,
   v_f == v_i + a*t,
]
print ("Kinematic equations:")
print (equations)
# Given v_i, v_f and a, find d
problem = [
    v_i == 30,
    v_f == 0,
    a   == -8
]
print ("Problem:")
print (problem)
print ("Solution:")
solve(equations + problem)

# Problem 2
# Ben Rushin is waiting at a stoplight. When it finally turns green,
# Ben accelerated from rest at a rate of a 6.00 m/s2 for a time of 4.10 seconds.
# Determine the displacement of Ben's car during this time period. 


equations = [
   d == v_i * t + (a*t**2)/2,
   v_f == v_i + a*t,
]
print ("Kinematic equations:")
print (equations)
problem = [
    v_i == 0,
    t == 4.10,
    a == 6.00
]
# given a , t , v_i  find d

print("Problem ")
print(problem)
print("Solution")
solve(equations+problem)



Kinematic equations:
[d == v__i*t + (a*t**2)/2, v__f == v__i + a*t]
Problem:
[v__i == 30, v__f == 0, a == -8]
Solution:
[a = -8, v__f = 0, v__i = 30, t = 3.75, d = 56.25]
Kinematic equations:
[d == v__i*t + (a*t**2)/2, v__f == v__i + a*t]
Problem 
[v__i == 0, t == 4.1, a == 6]
Solution
[a = 6, t = 4.1, v__i = 0, v__f = 24.6, d = 50.43]


---
# Puzzles

## Dog, Cat and Mouse

In [119]:
# Spend exactly 100 dollars and buy exactly 100 animals. 
# Dogs cost 15 dollars, cats cost 1 dollar, and mice cost 25 cents each.
# You have to buy at least one of each. How many of each should you buy? 

# Create 3 integer variables
dog, cat, mouse = Ints('dog cat mouse')
solve(dog >= 1,   # at least one dog
      cat >= 1,   # at least one cat
      mouse >= 1, # at least one mouse
      # we want to buy 100 animals
      dog + cat + mouse == 100,
      # We have 100 dollars (10000 cents):
      #   dogs cost 15 dollars (1500 cents), 
      #   cats cost 1 dollar (100 cents), and 
      #   mice cost 25 cents 
      1500 * dog + 100 * cat + 25 * mouse == 10000)

[cat = 41, mouse = 56, dog = 3]


## Sudoku

In [365]:


# 9x9 matrix of integer variables
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ]
      for i in range(9) ]

# each cell contains a value in {1, ..., 9}
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9)
             for i in range(9) for j in range(9) ]

# each row contains a digit at most once
rows_c   = [ Distinct(X[i]) for i in range(9) ]



# each column contains a digit at most once
cols_c   = [ Distinct([ X[i][j] for i in range(9) ])
             for j in range(9) ]



# each 3x3 square contains a digit at most once
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j]
                        for i in range(3) for j in range(3) ])
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c


# sudoku instance, we use '0' for empty cells
instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

def instance_c_generator(instance):
    return [ If(instance[i][j] == 0,
                  True,
                  X[i][j] == instance[i][j])
               for i in range(9) for j in range(9) ]

instance_c = instance_c_generator(instance)


s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(9) ]
    print_matrix(r)
else:
    print ("failed to solve")



[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]


In [347]:
import random
row_ind = [1,2,3,4,5,6,7,8,0]
col_ind = [1,2,3,4,5,6,7,8,0]
shuffle(row_ind)
shuffle(col_ind)

In [366]:
# Hardest Puzzle
ind = [0]*81
prev = -1 # for storing the last element
sat_found = False # for breaking the loops
s.reset() # remove all the constraints
s.add(sudoku_c)  # adding sudoku constraints ie. rows, cols and sub-matrix
negative_constraints = []
to_visit = 81 # number of elements to be visited
while(to_visit != 0):
    while (True) :
        num = random.randint(0,80)
        if(ind[num] == 0):
            break
    ind[num] = 1 #index is used
    i = int(num / 9)
    j = num % 9
    
    prev =  r[i][j] 
    r[i][j] = 0
    s.push() # for adding all negative constraints
    negative_constraints = negative_constraints + [ Not(X[i][j] == prev)]
    s.add([Or(negative_constraints)])
    s.push() # for adding the updated matrix elements
    s.add(instance_c_generator(r))
    #print("%d,%d" %(i,j))
    #print(s.check())
    if(s.check() == sat):
        r[i][j] = prev # restore the last element
        sat_found = True
    s.pop() # removing all negative constraints
    s.pop() # for removing last matrix
    to_visit = to_visit - 1 #update visited elements count
print_matrix(r)

[[0, 0, 5, 0, 0, 0, 0, 3, 0],
 [2, 3, 4, 5, 1, 0, 0, 0, 0],
 [0, 8, 0, 0, 0, 0, 0, 4, 0],
 [0, 9, 0, 0, 0, 7, 0, 0, 0],
 [0, 0, 7, 0, 0, 0, 9, 5, 4],
 [0, 0, 0, 4, 0, 0, 0, 0, 3],
 [0, 0, 0, 0, 0, 8, 0, 0, 9],
 [0, 2, 8, 0, 6, 0, 0, 0, 0],
 [5, 4, 0, 9, 0, 2, 0, 0, 0]]


In [367]:
#Testing the created Sudoku
s.reset()
s.add(sudoku_c + instance_c_generator(r))
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(9) ]
    print_matrix(r)
else:
    print ("failed to solve")

[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]


In [277]:
#Older sudoku generator
prev = -1
final_r = r
sat_found = False
constraints = sudoku_c
negative_constraints = []
for i in row_ind:
    for j in col_ind:
        prev =  r[i][j] 
        r[i][j] = 0
        s.reset()
        negative_constraints = negative_constraints +  [ Not(X[i][j] == prev)] 
        s.add(negative_constraints + sudoku_c +   instance_c_generator(r))
        print(s.check())
        if(s.check() == sat):
            r[i][j] = prev
            negative_constraints.pop()
            sat_found = True
            break
    if(sat_found):
        break

unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
sat
